<a href="https://colab.research.google.com/github/LujainOwaimer/Exercises/blob/main/RAG_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [1]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 9.46MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

In [36]:
!pip install langchain_huggingface

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [28]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq


  Using cached langchain_huggingface-0.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached langchain_community-0.3.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached Chroma-0.2.0.tar.gz (5.8 kB)
  Preparing metadata (setup.py) ... done
  Using cached langchain_groq-0.2.0-py3-none-any.whl.metadata (2.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━

In [41]:
import pandas as pd
import os
import markdown
from langchain.text_splitter import RecursiveCharacterTextSplitter
!pip install -U langchain-community
from langchain.embeddings import SentenceTransformerEmbeddings
!pip install chromadb
from langchain.vectorstores import Chroma

# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [42]:
df = pd.read_csv('/content/Dataset.csv')
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [64]:
directory = 'data/markdown_files'
os.makedirs(directory, exist_ok=True)

In [69]:
for i in range(0, len(df)):

    violation  = df['المخالفة'].iloc[i]
    fine  = df['الغرامة'].iloc[i]

    markdown_content = f"# {violation}\n\n"
    markdown_content += f"{fine}\n\n"

    with open(f'{directory}/{i}.md', 'w', encoding='utf-8') as file:
        file.write(markdown_content)

In [71]:
import markdown

markdown_texts = []
for filename in os.listdir(directory):
    if filename.endswith(".md"):
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            markdown_content = file.read()
            html_content = markdown.markdown(markdown_content)
            markdown_texts.append(html_content)

In [74]:
markdown_texts[0]

'<h1>طمس لوحات المركبة.</h1>\n<p>الغرامة المالية 3000 - 6000 ريال</p>'

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [76]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=30,
)

splitted_text = text_splitter.create_documents(markdown_texts)

In [77]:
splitted_text

[Document(metadata={}, page_content='<h1>طمس لوحات المركبة.</h1>\n<p>الغرامة المالية 3000 - 6000 ريال</p>'),
 Document(metadata={}, page_content='<h1>قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.</h1>\n<p>الغرامة المالية 100 - 150 ريال</p>'),
 Document(metadata={}, page_content='<h1>عدم ربط حزام الأمان.</h1>\n<p>الغرامة المالية 150 - 300 ريال</p>'),
 Document(metadata={}, page_content='<h1>استعمال المركبة لغير الغرض الذي رخصت من أجله.</h1>\n<p>الغرامة المالية 500 - 900 ريال</p>'),
 Document(metadata={}, page_content='<h1>القيادة على أكتاف الطريق وعلى الأرصفة أو المسارات التي يمنع القيادة فيها.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>'),
 Document(metadata={}, page_content='<h1>عدم قيام السائق الذي يرغب في تغيير مساره بإعطاء الأفضلية لسائق المركبة التي تسير في اتجاه مستقيم في حال سير المركبتين متحاذيتين بشكل متواز.</h1>\n<p>الغرامة المالية 500 - 900 ريال</p>'),
 Document(metadata={}, page_content='<h1>نقل الركاب في الأماكن غير المخصصة لهم في المركبة.</h1>\n<p>الغرامة المالية

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [54]:
model_name = "Omartificial-Intelligence-Space/GATE-AraBert-v0"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

embeddings = HuggingFaceEmbeddings(model_name=model_name)

vector_store = Chroma.from_texts(chunks, embeddings)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.50k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/672 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/761k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [80]:

from langchain_community.vectorstores import Chroma

db = Chroma(
    embedding_function=hf,
    persist_directory="./chroma_langchain_db",
)

<ipython-input-80-0d68ea26c338>:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  db = Chroma(


In [81]:
db.add_documents(splitted_text)
db.persist()

<ipython-input-81-0bab5f45db6e>:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [82]:
from langchain_core.prompts import PromptTemplate

PROMPT_TEMPLATE="""
give advice and answer the question  in arabic only based only on the context:
Context: {context}
Question: {question}
your advice:
"""

prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [83]:
!pip install openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.0/375.0 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.4 MB/s eta 0:00:00


In [84]:
!pip install langchain_groq -q

In [56]:
from google.colab import userdata
userdata.get('lujain')

'2147'

In [59]:
groq_api_key = "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"

In [85]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0.2,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=groq_api_key
)

In [86]:
from langchain.chains import LLMChain
from langchain_community.llms import OpenAI

model2 = LLMChain(llm=OpenAI(), prompt=prompt_template)

<ipython-input-86-909fb86719c7>:4: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  model2 = LLMChain(llm=OpenAI(), prompt=prompt_template)
<ipython-input-86-909fb86719c7>:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  model2 = LLMChain(llm=OpenAI(), prompt=prompt_template)


## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [87]:
from langchain.chains import LLMChain

model = LLMChain(llm=llm, prompt=prompt_template)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [88]:
def query_rag(query: str):
    similarity_search_results = db.similarity_search_with_score(query, k=1)
    context_text = ([doc.page_content for doc, _score in similarity_search_results])

    rag_response = model.invoke({"context": context_text, "question": query})

    return rag_response

In [89]:
def query_rag_with_model2(query: str):
    similarity_search_results = db.similarity_search_with_score(query, k=1)
    context_text = ([doc.page_content for doc, _score in similarity_search_results])

    rag_response = model2.invoke({"context": context_text, "question": query})

    return rag_response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [90]:
respon = query_rag('ماهي الغرامة على القيادة بدون رخصة؟')
respon

{'context': ['<h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>'],
 'question': 'ماهي الغرامة على القيادة بدون رخصة؟',
 'text': 'النص الذي تم تقديمه لي لا يتحدد فيه غرامة على القيادة بدون رخصة، ولكن يتحدد فيه غرامة على قيادة المركبة برخصة لا تتناسب مع حجم المركبة ونوع استخدامها، وهي تبلغ ما بين 1000 و 2000 ريال.\n\nيجب على السياقين احترام القوانين والأنظمة المقدمة من قبل الحكومات للحفاظ على سلامة المرور والمجتمع. ويجب عليهم اتباع الشروط والأحكام المتعلقة بالحصول على رخصة قيادة تتناسب مع نوع المركبة التي يرغبون في قيادتها.\n\nغرامة القيادة بدون رخصة قد تكون أكبر من هذه الغرامة، وقد يتم إتخاذ إجراءات أخرى مثل الإحباط أو الحبس في بعض البلدان. لذلك، يشير هذا النص إلى أهمية الحصول على رخصة قيادة صحيحة قبل قيادة أي مركبة.'}

In [94]:
respon = query_rag_with_model2('ماهي الغرامة على القيادة بدون رخصة؟')
respon

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [92]:
response = query_rag('ماهي الغرامة التباطؤ في السير على نحو يعرقل الحركة؟')
response

{'context': ['<h1>التباطؤ في السير على نحو يعرقل الحركة.</h1>\n<p>الغرامة المالية 300 - 500 ريال</p>'],
 'question': 'ماهي الغرامة التباطؤ في السير على نحو يعرقل الحركة؟',
 'text': 'الغرامة المالية للتباطؤ في السير على نحو يعرقل الحركة تبلغ ما بين 300 و 500 ريال ، كما هو موضح في السياق المقدم. يجب على الجميع الاحترام للسرعات المحددة والتي تؤثر في سلامة المرور والحركة النقلية.\n\nMy advice is to always follow the specified speed limits and be mindful of how your driving may affect the flow of traffic and overall safety of transportation.'}